In [ ]:
!pip install Afinn
!pip install configparser
!pip install pandas
!pip install textblob
!pip install flair
import tweepy
import pandas as pd
import numpy as np
import requests
import os
import re
import csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
input = "/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_final_5.0:3.0:3.0/test_dataset.csv"
output_folder = "/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_final_5.0:3.0:3.0"
name = "test_dataset_with_Scores"
name_of_tweet_column = "text"
sentiment_threshold = 0.001
save = True

In [ ]:
##helper functions

def determine_sent_from_score(score):
  if score > sentiment_threshold:
    return "Positive"
  elif score < (0 - sentiment_threshold) :
    return "Negative"
  else:
    return "Neutral"

In [ ]:
output_dir = output_folder + '/' + name + '.csv'
df_o = pd.read_csv(input)
display(df_o)
df_o.replace("", np.nan, inplace=True) # replace empty string with np.nan
df_o.dropna(subset = [name_of_tweet_column], inplace=True) # drop nan values
display(df_o)

In [ ]:
df = df_o.copy()

In [ ]:
##random
import secrets
def rand_choice():
  return secrets.choice(["Positive","Negative","Neutral"])

ls = []

for i in range(0,df.shape[0]):
  ls.append(rand_choice())

df['random_sent'] = ls

##choose only one label
df['Negative_sent'] = "Negative"
df['Positive_sent'] = "Positive"
df['Neutral_sent'] = "Neutral"

In [ ]:
##afinn lexicon
from afinn import Afinn
af = Afinn()

df.dropna(subset = [name_of_tweet_column], inplace=True)

score_df = []

for sentence in df[name_of_tweet_column]:
    score = af.score(sentence)
    #print(f'The sentiment value of the sentence :"{sentence}" is : {score}')
    score_df.append(score)
    #print(score)

df_score = pd.DataFrame(score_df,columns=['score'] )
display(df_score)

In [ ]:
#normalize affine scores to between -1 to 1 and add to df

# copy the data
df_max_scaled = df_score.copy()
  
# apply normalization techniques on Column 1
column = 'score'
df_max_scaled[column] = df_max_scaled[column] /df_max_scaled[column].abs().max()
  
# view normalized data
#display(df_max_scaled)

df['afinn_score'] = df_max_scaled['score']

df['afinn_sent'] = df_max_scaled['score'].apply(determine_sent_from_score)

display(df)

In [ ]:
##vader

!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer



In [ ]:
# init the sentiment analyzer
sia = SentimentIntensityAnalyzer()
score_df = []
#display(df.insert('score'))

for sentence in df[name_of_tweet_column]:
    score = sia.polarity_scores(sentence)["compound"]
    #print(f'The sentiment value of the sentence :"{sentence}" is : {score}')
    score_df.append(score)
    #print(score)

df_score = pd.DataFrame(score_df,columns=['score'] )
display(df_score)


In [ ]:
display(df)
df['vader_score'] = df_score['score']
df['vader_sent'] = df_score['score'].apply(determine_sent_from_score)
display(df)

In [ ]:
from nltk.tokenize import word_tokenize, RegexpTokenizer

import pandas as pd

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


analyzer = SentimentIntensityAnalyzer()
sentence = "quake"

new_words = {
    'bull': 2,
    'bear': -1.2,
    'bullish': 2,
    'bearish': -1.2,
    'dump': -1.5

}

analyzer.lexicon.update(new_words)
#s = analyzer.polarity_scores(sentence)
#print(s)

# init the sentiment analyzer
#sia = SentimentIntensityAnalyzer()
score_df = []
#display(df.insert('score'))

for sentence in df[name_of_tweet_column]:
    score = analyzer.polarity_scores(sentence)["compound"]
    #print(f'The sentiment value of the sentence :"{sentence}" is : {score}')
    score_df.append(score)
    #print(score)

df_score = pd.DataFrame(score_df,columns=['score'] )
display(df_score)

display(df)
df['modified_vader_score'] = df_score['score']
df['modified_vader_sent'] = df_score['score'].apply(determine_sent_from_score)
display(df)


In [ ]:
!pip install textblob

In [ ]:
##textblob

from textblob import TextBlob

df.dropna(subset = [name_of_tweet_column], inplace=True)

score_df = []
#display(df.insert('score'))
bob = df[name_of_tweet_column].head()
print(type(bob))
for sent in bob:
  typet = type(sent)

print(typet)

for sentence in df[name_of_tweet_column]:
    if type(sentence) != typet:
      print(type(sentence))
      print(sentence)
    res = TextBlob(sentence)
    score = res.sentiment.polarity
    #print(f'The sentiment value of the sentence :"{sentence}" is : {score}')
    score_df.append(score)
    #print(score)

df_score = pd.DataFrame(score_df,columns=['text_blob_score'] )
display(df_score)


<class 'pandas.core.series.Series'>
<class 'str'>


,text_blob_score
0,0.000000
1,0.500000
2,0.383333
3,0.383333
4,0.383333
...,...
833,0.383333
834,0.383333
835,0.400000
836,0.355804


In [ ]:
display(df)
df['textblob_score'] = df_score['text_blob_score']
df['textblob_sent'] = df_score['text_blob_score'].apply(determine_sent_from_score)


display(df)


df.to_csv(output_folder + "/" + name + "_lex" + ".csv")

,Unnamed: 0,text,label,source,quality,random_sent,Negative_sent,Positive_sent,Neutral_sent,afinn_score,afinn_sent,vader_score,vader_sent,modified_vader_score,modified_vader_sent
0,825,LEOPARD TOKEN🚀 🎯🎯 ✅Liquidity is locked ✅Certi...,Neutral,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,Neutral,0.0000,Neutral,0.0000,Neutral
1,323,$FTM is DIPPING! (z.score=-2.15) @ $2.47 11.4%...,Negative,self_manual,high,Positive,Negative,Positive,Neutral,0.000000,Neutral,-0.3595,Negative,-0.3595,Negative
2,606,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Positive,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive
3,548,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive
4,509,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Neutral,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,530,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive
834,660,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Positive,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive
835,780,💰💰Full thanks to as they have analysed this B...,Neutral,new_turk_high,high,Positive,Negative,Positive,Neutral,0.000000,Neutral,0.4655,Positive,0.4655,Positive
836,340,A heartly thanks to for giving me right tips...,Positive,new_turk_high,high,Neutral,Negative,Positive,Neutral,0.588235,Positive,0.9298,Positive,0.9298,Positive


,Unnamed: 0,text,label,source,quality,random_sent,Negative_sent,Positive_sent,Neutral_sent,afinn_score,afinn_sent,vader_score,vader_sent,modified_vader_score,modified_vader_sent,textblob_score,textblob_sent
0,825,LEOPARD TOKEN🚀 🎯🎯 ✅Liquidity is locked ✅Certi...,Neutral,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,Neutral,0.0000,Neutral,0.0000,Neutral,0.000000,Neutral
1,323,$FTM is DIPPING! (z.score=-2.15) @ $2.47 11.4%...,Negative,self_manual,high,Positive,Negative,Positive,Neutral,0.000000,Neutral,-0.3595,Negative,-0.3595,Negative,0.500000,Positive
2,606,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Positive,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive,0.383333,Positive
3,548,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive,0.383333,Positive
4,509,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Neutral,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive,0.383333,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,530,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive,0.383333,Positive
834,660,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Positive,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive,0.383333,Positive
835,780,💰💰Full thanks to as they have analysed this B...,Neutral,new_turk_high,high,Positive,Negative,Positive,Neutral,0.000000,Neutral,0.4655,Positive,0.4655,Positive,0.400000,Positive
836,340,A heartly thanks to for giving me right tips...,Positive,new_turk_high,high,Neutral,Negative,Positive,Neutral,0.588235,Positive,0.9298,Positive,0.9298,Positive,0.355804,Positive


In [ ]:
#flair
!pip install flair
!pip install pytorch
from flair.models import TextClassifier
from flair.data import Sentence


data_file_o = df.copy()
data_file_o.replace("", np.nan, inplace=True) # replace empty string with np.nan
data_file_o.dropna(subset = [name_of_tweet_column], inplace=True) # drop nan values
import pandas as pd
import flair
from flair.models import TextClassifier
from flair.data import Sentence



data_file = data_file_o.copy()

sia = TextClassifier.load('en-sentiment')


def sentiment_Flair(x):
    if x == '' or x == "" or x == np.nan:
      print("done")
      return np.nan,np.nan
    else:
      try:
        #print(x)
        sample = Sentence(x)
        sia.predict(sample)
        #print(sample)
        #print(i)
        #print(".")
        #print(sample.labels[0])
        value = sample.labels[0].value
        if value == "Positive":
          score_num = 1
        elif value == "NegativeE":
          score_num = -1
        elif value == "Neutral":
            score_num = 0
        else:
            value = np.nan
    
        return score_num ,sample.labels[0].score,
      except:
        return np.nan,np.nan


df_label = []
df_conf = []

for tweet in data_file[name_of_tweet_column]:
  label,conf = sentiment_Flair(tweet)
  df_label.append(label)
  df_conf.append(conf)



data_file['flair_label'] = df_label
data_file['flair_conf'] = df_conf
data_file['flair_sent'] =  data_file['flair_label'].apply(determine_sent_from_score)
display(data_file)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-x_jamk_r/pytorch_e4f7624b034f43c98ee3a0aa35ebd645/setup.py'"'"'; __file__='"'"'/tmp/pip-install-x_jamk_r/pytorch_e4f7624b034f43c98ee3a0aa35ebd645/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-recor

,Unnamed: 0,text,label,source,quality,random_sent,Negative_sent,Positive_sent,Neutral_sent,afinn_score,afinn_sent,vader_score,vader_sent,modified_vader_score,modified_vader_sent,textblob_score,textblob_sent,flair_label,flair_conf,flair_sent
0,825,LEOPARD TOKEN🚀 🎯🎯 ✅Liquidity is locked ✅Certi...,Neutral,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,Neutral,0.0000,Neutral,0.0000,Neutral,0.000000,Neutral,NaN,NaN,Neutral
1,323,$FTM is DIPPING! (z.score=-2.15) @ $2.47 11.4%...,Negative,self_manual,high,Positive,Negative,Positive,Neutral,0.000000,Neutral,-0.3595,Negative,-0.3595,Negative,0.500000,Positive,NaN,NaN,Neutral
2,606,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Positive,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive,0.383333,Positive,NaN,NaN,Neutral
3,548,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive,0.383333,Positive,NaN,NaN,Neutral
4,509,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Neutral,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive,0.383333,Positive,NaN,NaN,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,530,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive,0.383333,Positive,NaN,NaN,Neutral
834,660,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Positive,Negative,Positive,Neutral,0.000000,Neutral,0.6845,Positive,0.6845,Positive,0.383333,Positive,NaN,NaN,Neutral
835,780,💰💰Full thanks to as they have analysed this B...,Neutral,new_turk_high,high,Positive,Negative,Positive,Neutral,0.000000,Neutral,0.4655,Positive,0.4655,Positive,0.400000,Positive,NaN,NaN,Neutral
836,340,A heartly thanks to for giving me right tips...,Positive,new_turk_high,high,Neutral,Negative,Positive,Neutral,0.588235,Positive,0.9298,Positive,0.9298,Positive,0.355804,Positive,NaN,NaN,Neutral


In [ ]:
!pip install transformers
!pip install datasets
import requests
import os
import re
import csv
from transformers import pipeline
import torch.nn.functional as F
from transformers import AutoTokenizer , AutoModelForSequenceClassification
import torch




In [ ]:
def huggingface_model_out(directory,tweets):

  tokenizer = AutoTokenizer.from_pretrained(directory)
  model = AutoModelForSequenceClassification.from_pretrained(directory)
  classifier = pipeline("sentiment-analysis", model = model, tokenizer = tokenizer)
  df = pd.DataFrame(columns=['label','conf'])
  label_list = []
  #conf = []
  score_ = []
  for tweet in tweets:
    res = classifier(tweet)[0]
    lab = res['label']
    #label.append(lab)
    #conf.append(res['score'])
    if lab == "Positive" or lab == "POS" or lab == "LABEL_2" or lab == "positive"  :
      label = "Positive"
      score = 1
    elif lab == "Negative" or lab == "NEG" or lab == "LABEL_0" or lab == "negative":
      label = "Negative"
      score = -1
    elif lab == "Neutral" or lab == "NEU" or lab == "LABEL_1" or lab == "neutral":
      label = "Neutral"
      score = 0
    else:
      print(name," error model" , label)
      score = np.nan
    label_list.append(label)  
    score_.append(score)

  return label_list,score_


model_name = "svalabs/twitter-xlm-roberta-bitcoin-sentiment"


#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSequenceClassification.from_pretrained(model_name)
#classifier = pipeline("sentiment-analysis", model = model, tokenizer = tokenizer)
#res = classifier(df.tweet.values.tolist())

#label,conf,score = huggingface_model_out(model_name,data_file[name_of_tweet_column])
#result = huggingface_model_out(model_name,df['tweet'])
#display(score)
#display(label)

#display(result)


def hugging_face_sentiment_input(df,model_name,name):
  name = model_name
  label,score = huggingface_model_out(model_name,data_file[name_of_tweet_column])
  df[name + '_score'] = score
  df[name + '_sent'] = label
  #df[name + '_conf'] = conf
  return df






In [ ]:
#data_file['xlm_model_score'] = score
#data_file['xlm_model_label'] = label
#data_file['xlm_model_conf'] = conf
data_file = hugging_face_sentiment_input(data_file,"svalabs/twitter-xlm-roberta-bitcoin-sentiment","xlm_model")
data_file = hugging_face_sentiment_input(data_file,"finiteautomata/bertweet-base-sentiment-analysis","finiteautomata_bert")
data_file = hugging_face_sentiment_input(data_file,"cardiffnlp/bertweet-base-sentiment","cardiffnlp_bert")
data_file = hugging_face_sentiment_input(data_file,"l3cube-pune/MarathiSentiment","l3cube_marathi")
data_file = hugging_face_sentiment_input(data_file,"Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment","Tomas23_roberta")
data_file = hugging_face_sentiment_input(data_file,"elozano/tweet_sentiment_eval","elozano")
data_file = hugging_face_sentiment_input(data_file,"amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061","amansolanki")
data_file = hugging_face_sentiment_input(data_file,"cardiffnlp/twitter-xlm-roberta-base-sentiment","cardiff/roberta_xlm")


#data_file = hugging_face_sentiment_input(data_file,"amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061","amansolanki")

#data_file = hugging_face_sentiment_input(data_file,"elozano/tweet_sentiment_eval","elozano")
display(data_file)

,Unnamed: 0,text,label,source,quality,random_sent,Negative_sent,Positive_sent,Neutral_sent,afinn_score,...,l3cube-pune/MarathiSentiment_score,l3cube-pune/MarathiSentiment_sent,Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment_score,Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment_sent,elozano/tweet_sentiment_eval_score,elozano/tweet_sentiment_eval_sent,amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061_score,amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061_sent,cardiffnlp/twitter-xlm-roberta-base-sentiment_score,cardiffnlp/twitter-xlm-roberta-base-sentiment_sent
0,825,LEOPARD TOKEN🚀 🎯🎯 ✅Liquidity is locked ✅Certi...,Neutral,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,...,0,Neutral,0,Neutral,0,Neutral,0,Neutral,0,Neutral
1,323,$FTM is DIPPING! (z.score=-2.15) @ $2.47 11.4%...,Negative,self_manual,high,Positive,Negative,Positive,Neutral,0.000000,...,0,Neutral,1,Positive,1,Positive,-1,Negative,-1,Negative
2,606,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Positive,Negative,Positive,Neutral,0.000000,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
3,548,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
4,509,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Neutral,Negative,Positive,Neutral,0.000000,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,530,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
834,660,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Positive,Negative,Positive,Neutral,0.000000,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
835,780,💰💰Full thanks to as they have analysed this B...,Neutral,new_turk_high,high,Positive,Negative,Positive,Neutral,0.000000,...,0,Neutral,1,Positive,1,Positive,1,Positive,1,Positive
836,340,A heartly thanks to for giving me right tips...,Positive,new_turk_high,high,Neutral,Negative,Positive,Neutral,0.588235,...,1,Positive,1,Positive,1,Positive,1,Positive,1,Positive


In [ ]:
#data_file = hugging_face_sentiment_input(data_file,"","")
#data_file = hugging_face_sentiment_input(data_file,"","")
display(data_file)

,Unnamed: 0,text,label,source,quality,random_sent,Negative_sent,Positive_sent,Neutral_sent,afinn_score,...,l3cube-pune/MarathiSentiment_score,l3cube-pune/MarathiSentiment_sent,Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment_score,Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment_sent,elozano/tweet_sentiment_eval_score,elozano/tweet_sentiment_eval_sent,amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061_score,amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061_sent,cardiffnlp/twitter-xlm-roberta-base-sentiment_score,cardiffnlp/twitter-xlm-roberta-base-sentiment_sent
0,825,LEOPARD TOKEN🚀 🎯🎯 ✅Liquidity is locked ✅Certi...,Neutral,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,...,0,Neutral,0,Neutral,0,Neutral,0,Neutral,0,Neutral
1,323,$FTM is DIPPING! (z.score=-2.15) @ $2.47 11.4%...,Negative,self_manual,high,Positive,Negative,Positive,Neutral,0.000000,...,0,Neutral,1,Positive,1,Positive,-1,Negative,-1,Negative
2,606,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Positive,Negative,Positive,Neutral,0.000000,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
3,548,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
4,509,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Neutral,Negative,Positive,Neutral,0.000000,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,530,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Negative,Negative,Positive,Neutral,0.000000,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
834,660,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Positive,Negative,Positive,Neutral,0.000000,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
835,780,💰💰Full thanks to as they have analysed this B...,Neutral,new_turk_high,high,Positive,Negative,Positive,Neutral,0.000000,...,0,Neutral,1,Positive,1,Positive,1,Positive,1,Positive
836,340,A heartly thanks to for giving me right tips...,Positive,new_turk_high,high,Neutral,Negative,Positive,Neutral,0.588235,...,1,Positive,1,Positive,1,Positive,1,Positive,1,Positive


In [ ]:
#save = True
if save:
  data_file.to_csv(output_dir)


In [ ]:
df_check = pd.read_csv(output_dir)
display(df_check)

,Unnamed: 0,Unnamed: 0.1,text,label,source,quality,random_sent,Negative_sent,Positive_sent,Neutral_sent,...,l3cube-pune/MarathiSentiment_score,l3cube-pune/MarathiSentiment_sent,Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment_score,Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment_sent,elozano/tweet_sentiment_eval_score,elozano/tweet_sentiment_eval_sent,amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061_score,amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061_sent,cardiffnlp/twitter-xlm-roberta-base-sentiment_score,cardiffnlp/twitter-xlm-roberta-base-sentiment_sent
0,0,825,LEOPARD TOKEN🚀 🎯🎯 ✅Liquidity is locked ✅Certi...,Neutral,incomplete_valid,high,Negative,Negative,Positive,Neutral,...,0,Neutral,0,Neutral,0,Neutral,0,Neutral,0,Neutral
1,1,323,$FTM is DIPPING! (z.score=-2.15) @ $2.47 11.4%...,Negative,self_manual,high,Positive,Negative,Positive,Neutral,...,0,Neutral,1,Positive,1,Positive,-1,Negative,-1,Negative
2,2,606,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Positive,Negative,Positive,Neutral,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
3,3,548,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Negative,Negative,Positive,Neutral,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
4,4,509,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Neutral,Negative,Positive,Neutral,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,833,530,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Negative,Negative,Positive,Neutral,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
834,834,660,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high,Positive,Negative,Positive,Neutral,...,-1,Negative,1,Positive,1,Positive,1,Positive,1,Positive
835,835,780,💰💰Full thanks to as they have analysed this B...,Neutral,new_turk_high,high,Positive,Negative,Positive,Neutral,...,0,Neutral,1,Positive,1,Positive,1,Positive,1,Positive
836,836,340,A heartly thanks to for giving me right tips...,Positive,new_turk_high,high,Neutral,Negative,Positive,Neutral,...,1,Positive,1,Positive,1,Positive,1,Positive,1,Positive
